In [1]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
import pandas as pd
import numpy as np
import re
import math
import numpy as np

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/herick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/herick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 비슷한 이름 상품 찾기
## 수학에서 distance란?
거리 = 점과 점 사이의 거리 <br/>
질문 = 두 점 사이의 거리는 무엇으로 측정 되는가? meter? feet? 혹은 각도?

### **가장 자주 쓰이는 거리(기존에 우리가 알고 있는) : Elucidian Distnace**
<img src = "image_data/eul_distance.png">
**단점**
1. 데이터의 차원이 높아질수록 거리 계산이 부정확 해진다<br/>
2. 두 점 사이의 절대값이 차이가 많이 아지만, 이러한 절대값 차이가 유의미 하지 않을때<br/>
3. 두 점의 성질이 얼마나 다른지 알고 싶을떄<br/>

### **수학에서 자주 쓰이는 :cosine similarity**
<img src = "image_data/cos_distance.svg">
<img src = "image_data/cos_distance2.png">
1. 데이터의 차원이 높아질져도 먹힌다<br/>
2. 두 점 사이으이 절대값을 중요한 정보로 여기지 않는다<br/>
3. -1 과 1 사이의 값으로 standardize 되기 때문에 스코어로 활용하기 쉽다(혹은 확률에 적용시키기 쉽다)<br/>


## 잠깜만, -1과 1 사이, 즉 스코어로 활용하기 쉽다는 것은 무엇을 의미 할까? = 확률과 연동 시키기 쉽다

case1 = 점1 주변에 다른 점들이 무작위로 찍힌다고 가정 할 때 cosin similarity 스코어가 0과 1 사이일 확률은 몇% 일까?<br/>
case2 = 점1 주변에 다른 점들이 무작위로 찍힌다고 가정 할 때 Elucidian distance가 0과 max(무작위로 찍힌 점 까지의 절대 거리 값)/2 사이일 확률은 몇% 일까?<br/>

### 실질적으로 많은 machine learning method에서 cosin similarity 를 쓴다


# 문제 : 특정 상품과 비슷한 이름을 가지는 상품을 찾으시오

In [3]:
#https://archive.ics.uci.edu/ml/datasets/Online%20Retail
# df= pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
# df.to_excel('./data/retail_data.xlsx')

In [4]:
df = pd.read_excel('./data/retail_data.xlsx')

In [5]:
for i, x in enumerate(df['Description']):
    if i <20:
        print(x)

WHITE HANGING HEART T-LIGHT HOLDER
WHITE METAL LANTERN
CREAM CUPID HEARTS COAT HANGER
KNITTED UNION FLAG HOT WATER BOTTLE
RED WOOLLY HOTTIE WHITE HEART.
SET 7 BABUSHKA NESTING BOXES
GLASS STAR FROSTED T-LIGHT HOLDER
HAND WARMER UNION JACK
HAND WARMER RED POLKA DOT
ASSORTED COLOUR BIRD ORNAMENT
POPPY'S PLAYHOUSE BEDROOM 
POPPY'S PLAYHOUSE KITCHEN
FELTCRAFT PRINCESS CHARLOTTE DOLL
IVORY KNITTED MUG COSY 
BOX OF 6 ASSORTED COLOUR TEASPOONS
BOX OF VINTAGE JIGSAW BLOCKS 
BOX OF VINTAGE ALPHABET BLOCKS
HOME BUILDING BLOCK WORD
LOVE BUILDING BLOCK WORD
RECIPE BOX WITH METAL HEART


In [6]:
stop_words = set(stopwords.words('english')) 
def str_filter(x):
    if type(x) == str:
        
        x = x.strip(' ').lower().replace('-', ' ').replace('/' , ' ')
        x = re.sub('[0-9]', '', x)
        x = re.sub('\b\w{1}\b', '', x)
        
        stop_words = set(stopwords.words('english'))         
        filtered_product_name_set = set([w for w in x.split(' ') if not w in stop_words] )
        return filtered_product_name_set
    else:
        return None


df['Description_filtered'] = df['Description'].apply(lambda x : str_filter(x))


In [7]:
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description_filtered
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,"{hanging, light, heart, white, holder}"
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,"{white, metal, lantern}"
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,"{coat, hearts, cupid, hanger, cream}"
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,"{bottle, flag, knitted, hot, water, union}"
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,"{woolly, hottie, red, white, heart.}"


In [30]:
target_prd = df['Description_filtered'][0]
compare_prd_list = df['Description_filtered'][1:1000]

In [31]:
target_prd

{'hanging', 'heart', 'holder', 'light', 'white'}

In [32]:
compare_prd_list[:10]

1                        {white, metal, lantern}
2           {coat, hearts, cupid, hanger, cream}
3     {bottle, flag, knitted, hot, water, union}
4           {woolly, hottie, red, white, heart.}
5              {, nesting, set, babushka, boxes}
6          {frosted, light, holder, glass, star}
7                    {warmer, hand, union, jack}
8                {polka, dot, red, warmer, hand}
9             {bird, colour, assorted, ornament}
10                 {bedroom, playhouse, poppy's}
Name: Description_filtered, dtype: object

In [37]:
A = len(target_prd)
score_dic = dict()
for i, each_prd in enumerate(compare_prd_list):
    if each_prd != None:
        i_next = i +1
        AB = 0    
        B = len(each_prd)
        for each_word in each_prd:
            if each_word in target_prd:
                AB += 1

        score = round(AB/(math.sqrt(A) * math.sqrt(B)), 3)
        score_dic[i_next] = score




In [38]:
score_dic

{1: 0.258,
 2: 0.0,
 3: 0.0,
 4: 0.2,
 5: 0.0,
 6: 0.4,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.224,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.2,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 1.0,
 50: 0.258,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.183,
 58: 0.183,
 59: 0.2,
 60: 0.224,
 61: 0.0,
 62: 0.2,
 63: 0.0,
 64: 0.4,
 65: 0.0,
 66: 1.0,
 67: 0.258,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.183,
 75: 0.183,
 76: 0.2,
 77: 0.224,
 78: 0.0,
 79: 0.2,
 80: 0.0,
 81: 0.4,
 82: 0.0,
 83: 0.8,
 84: 0.0,
 85: 0.0,
 86: 0.0,
 87: 0.183,
 88: 0.0,
 89: 0.0,
 90: 0.0,
 91: 0.0,
 92: 0.0,
 93: 0.0,
 94: 0.0,
 95: 0.0,
 96: 0.0,
 97: 0.0,
 98: 0.0,
 99: 0.

In [39]:
compare_prd_list[1], compare_prd_list[19]

({'lantern', 'metal', 'white'}, {'box', 'heart', 'metal', 'recipe'})

In [43]:
for k in score_dic.keys():
    print(k, type(k))
    break

1 <class 'int'>


In [54]:
for k,v in score_dic.items():
    if v >= 0.5:
        print(k,v, compare_prd_list[k])

49 1.0 {'hanging', 'light', 'heart', 'white', 'holder'}
66 1.0 {'hanging', 'light', 'heart', 'white', 'holder'}
83 0.8 {'hanging', 'red', 'light', 'heart', 'holder'}
156 0.6 {'colour', 'hanging', 'light', 'holder', 'glass'}
217 0.8 {'hanging', 'zinc', 'light', 'heart', 'holder'}
220 1.0 {'hanging', 'light', 'heart', 'white', 'holder'}
262 1.0 {'hanging', 'light', 'heart', 'white', 'holder'}
278 1.0 {'hanging', 'light', 'heart', 'white', 'holder'}
364 1.0 {'hanging', 'light', 'heart', 'white', 'holder'}
416 1.0 {'hanging', 'light', 'heart', 'white', 'holder'}
450 0.6 {'zinc', 'light', 'heart', 'single', 'holder'}
451 0.8 {'hanging', 'red', 'light', 'heart', 'holder'}
453 0.671 {'holder', 'silver', 'hanging', 'light'}
816 0.775 {'holder', 'light', 'heart'}
861 0.775 {'holder', 'light', 'heart'}
935 1.0 {'hanging', 'light', 'heart', 'white', 'holder'}
979 1.0 {'hanging', 'light', 'heart', 'white', 'holder'}
